# *Summary*

## This notebook is an extension of the credit risk modelling from credit log modelling.

### In this notebook, the credit risk modelled with gradient boosted trees for predicting loan defaults with probability. This can be used to automate approving and declining loan applcations more accurately.

### An 93.3% accuracy level was achieved in predicting the loan defaults on 32,576 loans and 12 benchmarks with a probability threshold of 60%. With this model, the default rate would decrease from 21.8% to 6.7%, resulting in minimized risk for both the lender and applicant.
   
### The top 5 most important features of determining a loan default depends on the applicant's income, age, and employment length and the loan's interest rate and amount.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import model_selection,linear_model, metrics
import xgboost as xgb

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
cr_data = pd.read_csv("/kaggle/input/credit-risk-dataset/credit_risk_dataset.csv")

emp_len_null = cr_data[cr_data['person_emp_length'].isnull()].index
int_rate_null = cr_data[cr_data['loan_int_rate'].isnull()].index

cr_data['person_emp_length'].fillna((cr_data['person_emp_length'].median()), inplace=True)
cr_data['loan_int_rate'].fillna((cr_data['loan_int_rate'].median()), inplace = True)

cr_data = cr_data.rename(columns = {"cb_person_default_on_file":"default_hist", "cb_person_cred_hist_length": "cr_hist_len"})

cr_clean1 = cr_data[cr_data['person_age']<=100]

# one hot encoding categorical variables
num_col = cr_clean1.select_dtypes(exclude = 'object')
char_col = cr_clean1.select_dtypes(include = 'object')

encoded_char_col = pd.get_dummies(char_col)

cr_clean2 = pd.concat([num_col, encoded_char_col], axis=1)
cr_clean2

In [ ]:
# Split Train and Test Sets
Y = cr_clean2['loan_status']
X = cr_clean2.drop('loan_status',axis=1)
 


x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, random_state=2020, test_size=.30)

#Start of gradient boosted tree

xgb_model = xgb.XGBClassifier() # initialize tree

xgb_model.fit(x_train, np.ravel(y_train)) # train tree

predict_xgb = xgb_model.predict_proba(x_test) # 1st col = pred val, 2nd col = pred prob

predict_xgb_prob = pd.DataFrame(predict_xgb[:,1],columns = ['Default Probability'])

pd.concat([predict_xgb_prob, y_test.reset_index(drop=True)],axis=1)


In [ ]:
X

In [ ]:
print("There are {} features in cr_clean2".format(cr_clean2.shape[1]))
round(xgb_model.score(x_test,y_test),3)

In [ ]:
# display feature and their importance
feat_imp = xgb_model.get_booster().get_score(importance_type='weight')

feat_imp

In [ ]:
# columns not used
set(x_train.columns) - set(feat_imp)

The only feture not used was whether an applicant had a history of default.

In [ ]:
# display top 5 most import features
sorted(feat_imp.items(), key=lambda kv: kv[1],reverse=True)[0:5]

The top 5 most important features of determining a loan default depends on the applicant's income, age, and employment length and the loan's interest rate and amount.

In [ ]:
import operator as op
sorted_feat_imp = dict(sorted(feat_imp.items(), key=op.itemgetter(1),reverse=True))
sorted_feat_imp

In [ ]:
xgb.plot_importance(xgb_model,importance_type='weight')

In [ ]:

remove_feats = []

for key in feat_imp.keys():
    if feat_imp[key] < 40:    # tried 
        remove_feats.append(key)
        
imp_data = X
for key in remove_feats:
    imp_data = imp_data.drop(key,axis=1)

In [ ]:
imp_data.shape

The data has been reduced to 15 important features

In [ ]:
display_most_imp_feat ={}
for key in list(imp_data.columns):
    if key  in feat_imp.keys():
        display_most_imp_feat[key] = feat_imp[key]

sorted(display_most_imp_feat.items(), key=lambda kv: kv[1],reverse=True)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(imp_data, Y, random_state=2020, test_size=.30)

#Start of gradient boosted tree

xgb_model = xgb.XGBClassifier() # initialize tree

xgb_model.fit(x_train, np.ravel(y_train)) # train tree

predict_xgb = xgb_model.predict_proba(x_test) # 1st col = pred val, 2nd col = pred prob

predict_xgb_prob = pd.DataFrame(predict_xgb[:,1],columns = ['Default Probability'])

pd.concat([predict_xgb_prob, y_test.reset_index(drop=True)],axis=1)


In [ ]:
round(xgb_model.score(x_test,y_test),3)

In [ ]:
thresh = np.linspace(0,1,21)
thresh

In [ ]:
def find_opt_thresh(predict,thr =thresh, y_true = y_test):
    data = predict
    
    def_recalls = []
    nondef_recalls = []
    accs =[]

    
    for threshold in thr:
        # predicted values for each threshold
        data['loan_status'] = data['Default Probability'].apply(lambda x: 1 if x > threshold else 0 )
        
        accs.append(metrics.accuracy_score(y_true, data['loan_status']))
        
        stats = metrics.precision_recall_fscore_support(y_true, data['loan_status'])
        
        def_recalls.append(stats[1][1])
        nondef_recalls.append(stats[1][0])
        
        
    return accs, def_recalls, nondef_recalls

accs, def_recalls, nondef_recalls= find_opt_thresh(predict_xgb_prob)


In [ ]:
plt.plot(thresh,def_recalls)
plt.plot(thresh,nondef_recalls)
plt.plot(thresh,accs)
plt.xlabel("Probability Threshold")
plt.xticks(thresh, rotation = 'vertical')
plt.legend(["Default Recall","Non-default Recall","Model Accuracy"])
#plt.axvline(x=0.45, color='pink')
plt.show()

In [ ]:
optim_threshold = accs.index(max(accs))

print(round(accs[optim_threshold],3))

thresh[optim_threshold]

In [ ]:
cr_clean2.shape[1] - imp_data.shape[1]
# num of total features (including dummies) subtract important features with F score >= 40

In [ ]:
# original loan defaults by previous default history
default_hist_status_tab= pd.crosstab(cr_clean1['default_hist'], cr_clean1['loan_status'])
default_hist_status_tab

In [ ]:
round(default_hist_status_tab.iloc[:,1].sum() /cr_data.shape[0],3)

**The accuracy of the model was 93.3% with 15 important features and 12 feature removed once the data had been cleaned and encoded.**

**The gradient boosted tree performed 7.3% better than the logistics regression and had a higher probability threshold for default by 15%.**

**The initial loan default rate was 21.8%. With the `xgb_model`, the loan default rate should decrease to 6.7%. With this 15.1% improvement in default rate the lenders and applicants are even more so protected from risk.**